<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDIGOPNTS', 'INDUSINDBK', 'INDUSTOWER',
       'INFY', 'IOC', 'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO',
       'MARUTI', '

In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

L    67
H    31
M    27
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    70
1    55
Name: StarStock, dtype: int64

In [7]:
# star stocks
mypf['LatestQ'].value_counts()

0    85
1    40
Name: LatestQ, dtype: int64

In [8]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    110
1     15
Name: InPortfolio, dtype: int64

In [9]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [10]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [11]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQ'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQ'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [12]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-03-31 00:00:00+05:30,17360.0,^NSEI,-2.46,-0.24,-0.50,52.0,18812.0,15294.0,41.27,8.0
2023-04-11 00:00:00+05:30,17722.0,^NSEI,-0.44,-0.11,-0.30,63.0,18812.0,15294.0,30.98,6.0
2023-04-19 00:00:00+05:30,17619.0,^NSEI,-1.03,0.02,0.02,56.0,18812.0,15294.0,33.91,7.0
2023-04-26 00:00:00+05:30,17814.0,^NSEI,0.11,-0.03,-0.03,63.0,18812.0,15294.0,28.37,6.0
2023-05-04 00:00:00+05:30,18256.0,^NSEI,2.51,0.04,0.11,74.0,18812.0,15294.0,15.80,3.0


In [13]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [14]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [15]:
# portfolio stocks
cols = ['Close',	'Dev%_150',	'Slope%_150',	'Slope%_50',	'RSI',	'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQ']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 15


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQ
Stock,,,,,,,,,,,,,
INDIGOPNTS.NS,1198.0,-4.05,-2.24,0.47,74.0,87.33,123.0,1152.30,104.9,48.5,H,54.0,0
JUBLFOOD.NS,472.0,-7.11,-2.36,0.00,68.0,89.60,91.0,448.52,90.5,74.0,H,18.0,0
IRCTC.NS,632.0,-4.16,-0.91,-0.33,70.0,67.86,84.0,618.80,77.4,53.6,H,31.0,0
DMART.NS,3584.0,-5.87,-1.60,0.12,64.0,81.01,49.0,3501.37,160.1,99.2,H,38.0,0
INFY.NS,1274.0,-14.24,-0.81,-2.35,39.0,92.44,49.0,1252.12,29.0,22.0,H,24.0,1
BATAINDIA.NS,1505.0,-5.88,-1.69,-0.07,71.0,84.99,44.0,1495.89,126.0,60.5,H,52.0,0
PAGEIND.NS,40882.0,-5.18,-1.40,0.57,66.0,51.84,31.0,38201.00,97.7,66.7,H,32.0,0
GILLETTE.NS,4656.0,-3.74,-1.24,-0.78,71.0,73.27,28.0,4353.00,61.2,51.0,M,17.0,0
BAJFINANCE.NS,6390.0,-0.66,-1.38,-0.30,69.0,57.32,24.0,6044.41,52.6,33.6,H,36.0,1


In [16]:
tmp_df.sort_values(by = 'Discount%PE', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQ
Stock,,,,,,,,,,,,,
INDIGOPNTS.NS,1198.0,-4.05,-2.24,0.47,74.0,87.33,123.0,1152.30,104.9,48.5,H,54.0,0
BATAINDIA.NS,1505.0,-5.88,-1.69,-0.07,71.0,84.99,44.0,1495.89,126.0,60.5,H,52.0,0
DMART.NS,3584.0,-5.87,-1.60,0.12,64.0,81.01,49.0,3501.37,160.1,99.2,H,38.0,0
BAJFINANCE.NS,6390.0,-0.66,-1.38,-0.30,69.0,57.32,24.0,6044.41,52.6,33.6,H,36.0,1
PAGEIND.NS,40882.0,-5.18,-1.40,0.57,66.0,51.84,31.0,38201.00,97.7,66.7,H,32.0,0
IRCTC.NS,632.0,-4.16,-0.91,-0.33,70.0,67.86,84.0,618.80,77.4,53.6,H,31.0,0
INFY.NS,1274.0,-14.24,-0.81,-2.35,39.0,92.44,49.0,1252.12,29.0,22.0,H,24.0,1
EICHERMOT.NS,3339.0,1.10,-0.64,0.26,67.0,31.76,15.0,3275.70,44.7,34.8,H,22.0,0
JUBLFOOD.NS,472.0,-7.11,-2.36,0.00,68.0,89.60,91.0,448.52,90.5,74.0,H,18.0,0


In [17]:
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQ
Stock,,,,,,,,,,,,,
TATACONSUM.NS,776.0,2.99,-0.53,0.70,72.0,45.00,13.0,732.53,68.8,65.8,H,4.0,1
PAGEIND.NS,40882.0,-5.18,-1.40,0.57,66.0,51.84,31.0,38201.00,97.7,66.7,H,32.0,0
PIDILITIND.NS,2465.0,-1.51,-1.08,0.51,66.0,47.97,18.0,2442.09,94.6,101.0,H,-7.0,0
INDIGOPNTS.NS,1198.0,-4.05,-2.24,0.47,74.0,87.33,123.0,1152.30,104.9,48.5,H,54.0,0
ASIANPAINT.NS,2983.0,-0.17,-1.07,0.46,71.0,58.89,20.0,2856.73,87.9,74.9,H,15.0,0
EICHERMOT.NS,3339.0,1.10,-0.64,0.26,67.0,31.76,15.0,3275.70,44.7,34.8,H,22.0,0
HDFCBANK.NS,1728.0,8.80,0.69,0.25,71.0,0.00,0.0,1672.84,22.0,21.0,H,5.0,1
RELIANCE.NS,2448.0,0.00,-0.41,0.21,66.0,46.42,15.0,2382.54,29.1,24.8,H,15.0,1
DMART.NS,3584.0,-5.87,-1.60,0.12,64.0,81.01,49.0,3501.37,160.1,99.2,H,38.0,0


In [18]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [19]:
# all time high sales and profit TTM

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANIENT.NS,1911.0,-33.98,-4.11,0.46,56.0,4165.0,1194.0,75.87,118.0,1.0,0.0,M,0
LTIM.NS,4501.0,-1.51,-0.28,-0.48,52.0,7439.0,3735.0,79.32,65.0,1.0,0.0,M,1
ICICIGI.NS,1090.0,-5.07,-0.78,-0.55,52.0,1620.0,1060.0,94.64,49.0,1.0,0.0,M,1
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0
SBICARD.NS,797.0,1.34,-1.40,0.40,67.0,1133.0,673.0,73.04,42.0,1.0,0.0,M,1
TATAPOWER.NS,204.0,-3.39,-0.95,0.00,63.0,287.0,106.0,45.86,41.0,1.0,0.0,M,0
BERGEPAINT.NS,621.0,5.18,-0.51,0.85,69.0,858.0,540.0,74.53,38.0,1.0,0.0,M,0
CIPLA.NS,934.0,-9.63,-0.97,-1.21,61.0,1173.0,856.0,75.39,26.0,1.0,0.0,M,0
TCS.NS,3221.0,-0.50,0.31,-0.95,53.0,3886.0,2915.0,68.49,21.0,1.0,0.0,M,1


In [20]:
df_prec_dev[df_prec_dev.index.isin(star_stocks)]

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANIENT.NS,1911.0,-33.98,-4.11,0.46,56.0,4165.0,1194.0,75.87,118.0,1.0,0.0,M,0
ADANIGREEN.NS,946.0,-38.00,-6.16,3.28,55.0,2970.0,462.0,80.70,214.0,-1.0,-1.0,L,1
ASIANPAINT.NS,2983.0,-0.17,-1.07,0.46,71.0,3566.0,2576.0,58.89,20.0,1.0,1.0,H,0
AUBANK.NS,685.0,9.98,0.00,0.49,66.0,717.0,493.0,14.29,5.0,1.0,0.0,M,1
BAJAJ-AUTO.NS,4458.0,18.12,1.06,1.45,72.0,4498.0,2993.0,2.66,1.0,NaN,NaN,L,1
BAJAJFINSV.NS,1374.0,-7.57,-1.88,-0.38,63.0,1907.0,1093.0,65.48,39.0,0.0,0.0,M,1
BAJFINANCE.NS,6390.0,-0.66,-1.38,-0.30,69.0,7901.0,5265.0,57.32,24.0,1.0,1.0,H,1
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0
BERGEPAINT.NS,621.0,5.18,-0.51,0.85,69.0,858.0,540.0,74.53,38.0,1.0,0.0,M,0


In [21]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
BERGEPAINT.NS,621.0,5.18,-0.51,0.85,69.0,858.0,540.0,74.53,38.0,1.0,0.0,M,0
ADANIENT.NS,1911.0,-33.98,-4.11,0.46,56.0,4165.0,1194.0,75.87,118.0,1.0,0.0,M,0
SBICARD.NS,797.0,1.34,-1.40,0.40,67.0,1133.0,673.0,73.04,42.0,1.0,0.0,M,1
TATAPOWER.NS,204.0,-3.39,-0.95,0.00,63.0,287.0,106.0,45.86,41.0,1.0,0.0,M,0
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0
LTIM.NS,4501.0,-1.51,-0.28,-0.48,52.0,7439.0,3735.0,79.32,65.0,1.0,0.0,M,1
ICICIGI.NS,1090.0,-5.07,-0.78,-0.55,52.0,1620.0,1060.0,94.64,49.0,1.0,0.0,M,1
TCS.NS,3221.0,-0.50,0.31,-0.95,53.0,3886.0,2915.0,68.49,21.0,1.0,0.0,M,1
CIPLA.NS,934.0,-9.63,-0.97,-1.21,61.0,1173.0,856.0,75.39,26.0,1.0,0.0,M,0


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS','SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ACC.NS,1766.0,-18.08,-2.64,-0.57,53.0,2748.0,1614.0,86.60,56.0,0.0,0.0,L,0
AMBUJACEM.NS,388.0,-16.67,-1.93,1.08,54.0,588.0,275.0,63.90,52.0,0.0,0.0,L,0
PGHH.NS,13973.0,0.80,-0.22,0.19,52.0,15778.0,12000.0,47.78,13.0,0.0,0.0,L,0
NESTLEIND.NS,21694.0,11.34,0.86,1.37,77.0,21876.0,15919.0,3.06,1.0,1.0,1.0,H,0
SIEMENS.NS,3578.0,18.53,0.99,1.09,76.0,3578.0,1922.0,0.00,0.0,1.0,1.0,H,0


In [23]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,153.0,-12.69,-1.71,-0.66,60.0,303.0,137.0,90.36,98.0,1.0,1.0,H,1
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0


In [24]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,153.0,-12.69,-1.71,-0.66,60.0,303.0,137.0,90.36,98.0,1.0,1.0,H,1
HDFCLIFE.NS,546.0,1.58,-0.37,0.40,61.0,757.0,461.0,71.28,39.0,1.0,1.0,H,1
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0


In [25]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
SIEMENS.NS,3578.0,18.53,0.99,1.09,76.0,3578.0,1922.0,0.00,0.0,1.0,1.0,H,0
INDIANB.NS,332.0,21.88,2.94,1.38,67.0,334.0,112.0,0.90,1.0,1.0,1.0,H,0
NESTLEIND.NS,21694.0,11.34,0.86,1.37,77.0,21876.0,15919.0,3.06,1.0,1.0,1.0,H,0
BANKBARODA.NS,185.0,12.24,1.82,1.18,66.0,193.0,71.0,6.56,4.0,1.0,1.0,H,0
CANBK.NS,324.0,10.53,1.37,1.03,75.0,337.0,139.0,6.57,4.0,1.0,1.0,H,0
TITAN.NS,2670.0,5.01,-0.08,0.64,69.0,2770.0,1662.0,9.03,4.0,1.0,1.0,H,1
ICICIBANK.NS,922.0,4.26,0.00,0.69,69.0,953.0,619.0,9.28,3.0,1.0,1.0,H,1


In [26]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
NESTLEIND.NS,21694.0,11.34,0.86,1.37,77.0,21876.0,15919.0,3.06,1.0,1.0,1.0,H,0
SIEMENS.NS,3578.0,18.53,0.99,1.09,76.0,3578.0,1922.0,0.00,0.0,1.0,1.0,H,0


In [27]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 6


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
TITAN.NS,2670.0,5.01,-0.08,0.64,69.0,2770.0,1662.0,9.03,4.0,1.0,1.0,H,1
HDFCLIFE.NS,546.0,1.58,-0.37,0.40,61.0,757.0,461.0,71.28,39.0,1.0,1.0,H,1
INDUSINDBK.NS,1135.0,-1.17,-0.44,-0.09,57.0,1264.0,771.0,26.17,11.0,1.0,1.0,H,1
MARUTI.NS,8801.0,1.33,-0.39,-0.12,63.0,9528.0,6565.0,24.54,8.0,1.0,1.0,H,1
BANDHANBNK.NS,233.0,-0.68,-2.13,-0.46,70.0,346.0,187.0,71.07,48.0,1.0,1.0,L,0
INDUSTOWER.NS,153.0,-12.69,-1.71,-0.66,60.0,303.0,137.0,90.36,98.0,1.0,1.0,H,1


In [28]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 40


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANITRANS.NS,1012.0,-52.58,-9.38,-1.28,50.0,4105.0,643.0,89.34,306.0,1.0,-1.0,L,0
PEL.NS,758.0,-5.77,-2.11,-1.78,68.0,2866.0,635.0,94.49,278.0,-1.0,0.0,L,0
GLAND.NS,1372.0,-12.46,-4.02,0.23,59.0,4316.0,1154.0,93.11,215.0,1.0,0.0,M,0
ADANIGREEN.NS,946.0,-38.00,-6.16,3.28,55.0,2970.0,462.0,80.70,214.0,-1.0,-1.0,L,1
IBREALEST.NS,72.0,1.06,-1.41,3.39,71.0,191.0,47.0,82.64,165.0,-1.0,-1.0,L,0
ADANIENT.NS,1911.0,-33.98,-4.11,0.46,56.0,4165.0,1194.0,75.87,118.0,1.0,0.0,M,0
NETWORK18.NS,60.0,-3.95,-1.61,0.00,61.0,114.0,41.0,73.97,90.0,-1.0,-1.0,L,1
ZEEL.NS,195.0,-16.62,-2.14,-0.99,43.0,367.0,164.0,84.73,88.0,-1.0,1.0,L,0
WHIRLPOOL.NS,1342.0,-6.97,-1.87,0.30,57.0,2510.0,1254.0,92.99,87.0,1.0,0.0,M,0


In [29]:
stock_id = 'TITAN.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
TITAN.NS,2670.0,5.01,-0.08,0.64,69.0,2770.0,1662.0,9.03,4.0,1.0,1.0,H,1
